In [111]:
import numpy as np

In [94]:
links = {
    'webpage-1': set(['webpage-2', 'webpage-4', 'webpage-5', 'webpage-6', 'webpage-8', 'webpage-9', 'webpage-10']),
    'webpage-2': set(['webpage-5', 'webpage-6']),
    'webpage-3': set(['webpage-10']),
    'webpage-4': set(['webpage-9']),
    'webpage-5': set(['webpage-2', 'webpage-4']),
    'webpage-6': set([]), # dangling page
    'webpage-7': set(['webpage-1', 'webpage-3', 'webpage-4']),
    'webpage-8': set(['webpage-1']),
    'webpage-9': set(['webpage-1', 'webpage-2', 'webpage-3', 'webpage-8', 'webpage-10']),
    'webpage-10': set(['webpage-2', 'webpage-3', 'webpage-8', 'webpage-9']),
}

In [103]:
def build_idx(links):
    return {web:str(idx) for idx, web in enumerate(list(links.keys()))}

In [104]:
build_idx(links)

{'webpage-1': '0',
 'webpage-2': '1',
 'webpage-3': '2',
 'webpage-4': '3',
 'webpage-5': '4',
 'webpage-6': '5',
 'webpage-7': '6',
 'webpage-8': '7',
 'webpage-9': '8',
 'webpage-10': '9'}

In [128]:
def build_graph(links):
    links_idx = build_idx(links)
    graph = {}
    
    for key, val in links.items():
        graph[links_idx[key]] = [links_idx[v] for v in val]
    
    return graph

In [129]:
graph = build_graph(links)
graph

{'0': ['4', '5', '8', '9', '3', '1', '7'],
 '1': ['4', '5'],
 '2': ['9'],
 '3': ['8'],
 '4': ['1', '3'],
 '5': [],
 '6': ['2', '0', '3'],
 '7': ['0'],
 '8': ['2', '0', '9', '1', '7'],
 '9': ['2', '1', '8', '7']}

In [269]:
def build_matrix(graph):
    n = len(graph)
    matrix = np.zeros((n, n))
    
    for i in range(n):
        if not graph[str(i)]:
            matrix[i,:] = np.ones((1, n)) * 1 / n
        
        for j in graph[str(i)]:
            matrix[i][int(j)] = 1 / len(graph[str(i)])
    
    return matrix

In [270]:
A = build_matrix(graph)
A

array([[0.        , 0.14285714, 0.        , 0.14285714, 0.14285714,
        0.14285714, 0.        , 0.14285714, 0.14285714, 0.14285714],
       [0.        , 0.        , 0.        , 0.        , 0.5       ,
        0.5       , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.5       , 0.        , 0.5       , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.1       , 0.1       , 0.1       , 0.1       , 0.1       ,
        0.1       , 0.1       , 0.1       , 0.1       , 0.1       ],
       [0.33333333, 0.        , 0.33333333, 0.33333333, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.       

In [276]:
def pangrank(matrix, eps = 1.0e-8, d = 0.85):
    
    n = matrix.shape[0]
    R = np.ones((n, 1)) * 1 / n
    new_R = np.zeros((n, 1))
    E = np.ones((n, n))

    while True:
        new_R = ((1 - d) / n * E + d * matrix.T).dot(R)
        if np.linalg.norm(new_R - R) <= eps:
            break
        else:
            R = new_R
    
    return new_R

In [277]:
pangrank(A)

array([[0.13009588],
       [0.13050742],
       [0.08116303],
       [0.08539887],
       [0.09427651],
       [0.09427651],
       [0.0230135 ],
       [0.0904399 ],
       [0.13934097],
       [0.13148741]])

In [245]:
g = nx.DiGraph(graph)

In [278]:
#G = nx.DiGraph(nx.path_graph(4))
pr = nx.pagerank(g)
pr

{'0': 0.1300956900118335,
 '1': 0.13050776900162686,
 '2': 0.08116315448394983,
 '3': 0.08539910029620076,
 '4': 0.09427630133632156,
 '5': 0.09427630133632156,
 '6': 0.023013537818899983,
 '7': 0.09044007990712699,
 '8': 0.1393406839069631,
 '9': 0.13148738190075582}

In [ ]:
def sentence_similarity():
    
    pass